In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
#from apyori import apriori
from efficient_apriori import apriori

In [2]:
data = pd.read_excel("../Datasets/HVAC.xlsx", "HISTORICO_DATOS")

In [3]:
variables_a_eliminar = ["C_O_P_ BOMBA CALOR FELIPE", "C_O_P_ BOMBA CALOR CARLOS",
                        "C_O_P_ INSTALACIÓN GRUPO FRÍO 1", "C_O_P_ INSTALACÍON GRUPO FRÍO 2",
                        "ORDEN", "VÁLVULA BY PASS SECUNDARIO FRÍO",
                        "TEMPERATURA CONTROL DE BY PASS SECUNDARIO", "SECUNDARIO FRÍO 1",
                        "SECUNDARIO FRÍO 2", "SECUNDARIO FRÍO 3", "MODO INVIERNO BC1", 
                        "MODO INVIERNO BC2", "PERIODO P6", "CONTROL CALOR", 
                        "CAPACIDAD BOMBA CALOR FELIPE %", "CAPACIDAD BOMBA CALOR CARLOS %",
                        "CAPACIDAD GRUPO DE FRÍO 1"," CAPACIDAD GRUPO DE FRÍO 2",
                        "IMPULSIÓN SECUNDARIO CALOR", "SECUNDARIO CALOR 1", 
                        "SECUNDARIO CALOR 2", "SECUNDARIO CALOR 3"]
lista_variables = data.columns.values.tolist()
subLista = [x for x in lista_variables if x not in variables_a_eliminar]
data = data[subLista]

data = data
data["POTENCIA TERMICA BOMBA CALOR CARLOS"] = data["KILO CALORÍAS GENERADAS BOMBA CALOR CARLOS"] * 0.001163
data["POTENCIA TERMICA BOMBA CALOR FELIPE"] = data["KILO CALORÍAS GENERADAS BOMBA CALOR FELIPE"] * 0.001163
data["POTENCIA TERMICA GRUPO FRIO 1"] = data["KIGO FRIGORÍAS GENERADAS GRUPO DE FRÍO 1"] * 0.001163
data["POTENCIA TERMICA GRUPO FRIO 2"] = data["KIGO FRIGORÍAS GENERADAS GRUPO DE FRÍO 2"] * 0.001163

data.head()

,Fecha- hora de lectura,POTENCIA TRAFO 2,POTENCIA TRAFO 3,POTENCIA TRAFO 4,POTENCIA TRAFO 5,POTENCIA MEDIA CONECTADA,CONTROL FRÍO,POTENCIA BOMBA CALOR FELIPE,POTENCIA BOMBA CALOR CARLOS,TEMPERATURA AMBIENTE BOMBA CALOR CARLOS,...,TEMPERATURA SALIDA BOMBA CALOR CARLOS,TEMPERATURA SALIDA BOMBA CALOR FELIPE,KILO CALORÍAS GENERADAS BOMBA CALOR CARLOS,KILO CALORÍAS GENERADAS BOMBA CALOR FELIPE,KIGO FRIGORÍAS GENERADAS GRUPO DE FRÍO 1,KIGO FRIGORÍAS GENERADAS GRUPO DE FRÍO 2,POTENCIA TERMICA BOMBA CALOR CARLOS,POTENCIA TERMICA BOMBA CALOR FELIPE,POTENCIA TERMICA GRUPO FRIO 1,POTENCIA TERMICA GRUPO FRIO 2
0,2016-01-01 00:00:00,0.783603,38.338665,232.182602,235.755234,557.290039,26.984179,7.253334,7.061334,17.268667,...,36.190666,36.192665,28599.992188,26129.994141,4400.000488,29349.998047,33.261791,30.389183,5.117201,34.134048
1,2016-01-01 00:15:00,0.926969,38.338665,197.970230,113.384056,372.547760,26.805664,0.800000,0.480000,18.049334,...,33.625999,32.770664,163366.671875,193396.765625,18550.000000,0.000000,189.995439,224.920438,21.573650,0.000000
2,2016-01-01 00:30:00,0.793163,38.455196,211.676941,236.968918,469.299347,26.055836,0.800000,0.480000,17.046000,...,33.247997,32.559998,103306.765625,137973.359375,218650.000000,96750.000000,120.145768,160.463017,254.289950,112.520250
3,2016-01-01 00:45:00,0.860073,38.688263,206.066818,298.481995,570.994995,25.266325,0.800000,0.480000,13.052667,...,32.989998,32.559998,73883.296875,102829.898438,247150.000000,214850.000000,85.926274,119.591172,287.435450,249.870550
4,2016-01-01 01:00:00,0.812257,38.688263,193.984100,292.789215,563.804016,25.605207,0.800000,0.480000,10.656666,...,32.719997,32.242664,50396.769531,81683.398438,238550.000000,203300.000000,58.611443,94.997792,277.433650,236.437900


In [4]:
dataLabel = data
dataLabel["label_cop_carlos"] = (dataLabel["C_O_P BOMBA CALOR CARLOS"] > 3.5) & (dataLabel["C_O_P BOMBA CALOR CARLOS"] < 4.5)
dataLabel["label_cop_felipe"] = (dataLabel["C_O_P BOMBA CALOR FELIPE"] > 3.5) & (dataLabel["C_O_P BOMBA CALOR FELIPE"] < 4.5)
dataLabel["label_cop_frio_1"] = (dataLabel["C_O_P MÁQUINA GRUPO FRÍO 1"] > 3.5) & (dataLabel["C_O_P MÁQUINA GRUPO FRÍO 1"] < 4.5)
dataLabel["label_cop_frio_2"] = (dataLabel["C_O_P MÁQUINA GRUPO FRÍO 2"] > 3.5) & (dataLabel["C_O_P MÁQUINA GRUPO FRÍO 2"] < 4.5)
dataLabel.head()

,Fecha- hora de lectura,POTENCIA TRAFO 2,POTENCIA TRAFO 3,POTENCIA TRAFO 4,POTENCIA TRAFO 5,POTENCIA MEDIA CONECTADA,CONTROL FRÍO,POTENCIA BOMBA CALOR FELIPE,POTENCIA BOMBA CALOR CARLOS,TEMPERATURA AMBIENTE BOMBA CALOR CARLOS,...,KIGO FRIGORÍAS GENERADAS GRUPO DE FRÍO 1,KIGO FRIGORÍAS GENERADAS GRUPO DE FRÍO 2,POTENCIA TERMICA BOMBA CALOR CARLOS,POTENCIA TERMICA BOMBA CALOR FELIPE,POTENCIA TERMICA GRUPO FRIO 1,POTENCIA TERMICA GRUPO FRIO 2,label_cop_carlos,label_cop_felipe,label_cop_frio_1,label_cop_frio_2
0,2016-01-01 00:00:00,0.783603,38.338665,232.182602,235.755234,557.290039,26.984179,7.253334,7.061334,17.268667,...,4400.000488,29349.998047,33.261791,30.389183,5.117201,34.134048,False,False,False,False
1,2016-01-01 00:15:00,0.926969,38.338665,197.970230,113.384056,372.547760,26.805664,0.800000,0.480000,18.049334,...,18550.000000,0.000000,189.995439,224.920438,21.573650,0.000000,False,False,False,False
2,2016-01-01 00:30:00,0.793163,38.455196,211.676941,236.968918,469.299347,26.055836,0.800000,0.480000,17.046000,...,218650.000000,96750.000000,120.145768,160.463017,254.289950,112.520250,False,False,True,False
3,2016-01-01 00:45:00,0.860073,38.688263,206.066818,298.481995,570.994995,25.266325,0.800000,0.480000,13.052667,...,247150.000000,214850.000000,85.926274,119.591172,287.435450,249.870550,False,False,True,True
4,2016-01-01 01:00:00,0.812257,38.688263,193.984100,292.789215,563.804016,25.605207,0.800000,0.480000,10.656666,...,238550.000000,203300.000000,58.611443,94.997792,277.433650,236.437900,False,False,True,True


## Reglas de Asociación con el algoritmo Apriori

In [5]:
#rules = apriori(dataLabel, min_support = 0.003, min_confidence = 0.2, min_lift = 3, min_length = 2)

In [6]:
#results = list(rules)
#results

In [7]:
itemsets, rules = apriori(data, min_support=0.2,  min_confidence=1)

# Print out every rule with 2 items on the left hand side,
# 1 item on the right hand side, sorted by lift
rules_rhs = filter(lambda rule: len(rule.lhs) == 2 and len(rule.rhs) == 1, rules)
for rule in sorted(rules_rhs, key=lambda rule: rule.lift):
  print(rule) # Prints the rule and its confidence, support, lift, ...

{A, B} -> { } (conf: 1.000, supp: 0.324, lift: 1.121, conv: 108108108.108)
{A, C} -> { } (conf: 1.000, supp: 0.649, lift: 1.121, conv: 108108108.108)
{A, D} -> { } (conf: 1.000, supp: 0.324, lift: 1.121, conv: 108108108.108)
{A, F} -> { } (conf: 1.000, supp: 0.514, lift: 1.121, conv: 108108108.108)
{A, N} -> { } (conf: 1.000, supp: 0.622, lift: 1.121, conv: 108108108.108)
{A, O} -> { } (conf: 1.000, supp: 0.838, lift: 1.121, conv: 108108108.108)
{A, S} -> { } (conf: 1.000, supp: 0.324, lift: 1.121, conv: 108108108.108)
{A, T} -> { } (conf: 1.000, supp: 0.595, lift: 1.121, conv: 108108108.108)
{B, C} -> { } (conf: 1.000, supp: 0.324, lift: 1.121, conv: 108108108.108)
{B, E} -> { } (conf: 1.000, supp: 0.297, lift: 1.121, conv: 108108108.108)
{B, I} -> { } (conf: 1.000, supp: 0.297, lift: 1.121, conv: 108108108.108)
{B, L} -> { } (conf: 1.000, supp: 0.324, lift: 1.121, conv: 108108108.108)
{B, M} -> { } (conf: 1.000, supp: 0.324, lift: 1.121, conv: 108108108.108)
{B, O} -> { } (conf: 1.00

In [8]:
from efficient_apriori import apriori
transactions = [('eggs', 'bacon', 'soup'),
                ('eggs', 'bacon', 'apple'),
                ('soup', 'bacon', 'banana')]
itemsets, rules = apriori(transactions, min_support=0.2,  min_confidence=1)

# Print out every rule with 2 items on the left hand side,
# 1 item on the right hand side, sorted by lift
rules_rhs = filter(lambda rule: len(rule.lhs) == 2 and len(rule.rhs) == 1, rules)
for rule in sorted(rules_rhs, key=lambda rule: rule.lift):
  print(rule) # Prints the rule and its confidence, support, lift, ...

{apple, eggs} -> {bacon} (conf: 1.000, supp: 0.333, lift: 1.000, conv: 0.000)
{banana, soup} -> {bacon} (conf: 1.000, supp: 0.333, lift: 1.000, conv: 0.000)
{eggs, soup} -> {bacon} (conf: 1.000, supp: 0.333, lift: 1.000, conv: 0.000)
{apple, bacon} -> {eggs} (conf: 1.000, supp: 0.333, lift: 1.500, conv: 333333333.333)
{bacon, banana} -> {soup} (conf: 1.000, supp: 0.333, lift: 1.500, conv: 333333333.333)
